In [6]:
from sklearn import datasets
import sklearn
import pandas as pd

In [8]:
X,y = sklearn.datasets.fetch_covtype(return_X_y=True)

In [11]:
import numpy as np
file_path = 'output/MLP/Adult/predictions/p_1.npy'
content = np.load(file_path)
print(content)
pc = pd.DataFrame(content)
print(pc.head())

[[0.         0.98348832 0.01651166]
 [1.         0.75248456 0.24751541]
 [1.         0.41779459 0.58220541]
 ...
 [0.         0.6343646  0.36563542]
 [0.         0.81086361 0.1891364 ]
 [0.         0.6785121  0.32148793]]
     0         1         2
0  0.0  0.983488  0.016512
1  1.0  0.752485  0.247515
2  1.0  0.417795  0.582205
3  1.0  0.449932  0.550068
4  0.0  0.971243  0.028757


ModuleNotFoundError: No module named 'torch'